# Evaluación del Sistema de Recomendación
Análisis exhaustivo del sistema de recomendación usando datos sintéticos con todas las combinaciones posibles de habilidades blandas.

## Objetivo
Evaluar la precisión, recall y F1-score del sistema de recomendación generando 128 variaciones de perfiles de habilidades blandas por cada una de las 24 carreras, dando un total de **3,072 usuarios sintéticos**.

## Metodología
1. **Generación de datos sintéticos:** Crear todas las 2^7 = 128 combinaciones posibles de 7 habilidades blandas
2. **Asignación de vectores:** Asignar vectores técnicos base de cada carrera
3. **Expansión a 76 dimensiones:** Concatenar habilidades técnicas (69d) con blandas (7d)
4. **Vectorización de ofertas:** Cargar y vectorizar ofertas laborales por carrera
5. **Cálculo de similitud:** Generar recomendaciones basadas en similitud de coseno
6. **Creación de verdad fundamental:** Definir criterios de relevancia basados en habilidades blandas
7. **Métricas de evaluación:** Calcular precisión, recall y F1-score
8. **Análisis de resultados:** Agrupar por carrera y visualizar rendimiento

In [2]:
import pandas as pd
import numpy as np
import pickle
import os
import warnings
from itertools import product
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score

warnings.filterwarnings('ignore')

# Cargar datos preprocessados
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)

tfidf_epn_69d = datos['tfidf_epn_69d']
grupos_bge_ngram = datos['grupos_bge_ngram']
habilidades_whitelist = datos['habilidades']

## 1. Generación de datos sintéticos
Se crean todos los 2^7 = 128 combinaciones posibles de las 7 habilidades blandas para cada una de las 24 carreras disponibles.

### Configuración de habilidades blandas
Las 7 dimensiones de habilidades blandas son:
1. **Gestión**
2. **Comunicación efectiva**
3. **Liderazgo**
4. **Trabajo en equipo**
5. **Ética profesional**
6. **Responsabilidad social**
7. **Aprendizaje autónomo**

Cada combinación binaria (0 o 1) representa si el usuario tiene desarrollada esa habilidad o no.

In [3]:
# Definición de habilidades blandas
habilidades_blandas_config = {
    'Gestión': 0,
    'Comunicación efectiva': 1,
    'Liderazgo': 2,
    'Trabajo en equipo': 3,
    'Ética profesional': 4,
    'Responsabilidad social': 5,
    'Aprendizaje autónomo': 6
}

# Definición de carreras disponibles (mapeo de tfidf_epn_69d.T)
carreras_disponibles = list(tfidf_epn_69d.T.index)

print(f"Total de habilidades blandas: {len(habilidades_blandas_config)}")
print(f"Combinaciones posibles por carrera: 2^{len(habilidades_blandas_config)} = {2**len(habilidades_blandas_config)}")
print(f"Total de carreras: {len(carreras_disponibles)}")
print(f"Total de usuarios sintéticos: {(2**len(habilidades_blandas_config)) * len(carreras_disponibles)}")
print(f"\nCarreras:")
for carrera in carreras_disponibles:
    print(f"  - {carrera}")

Total de habilidades blandas: 7
Combinaciones posibles por carrera: 2^7 = 128
Total de carreras: 24
Total de usuarios sintéticos: 3072

Carreras:
  - Licenciatura Administracion De Empresas
  - Ingenieria Agroindustria
  - Ingenieria Ambiental
  - Ciencias De Datos E Inteligencia Artificial
  - Ingenieria En Ciencias De La Computacion
  - Economia
  - Ingenieria En Electricidad
  - Ingenieria En Electronica Y Automatizacion
  - Fisica
  - Ingenieria En Geologia
  - Ingenieria De La Produccion
  - Matematica
  - Matematica Aplicada
  - Ingenieria En Materiales
  - Ingenieria En Mecanica
  - Ingenieria En Mecatronica
  - Ingenieria En Petroleos
  - Ingenieria Quimica
  - Ingenieria En Sistemas De Informacion
  - Ingenieria En Software
  - Ingenieria En Telecomunicacion De La Informacion
  - Ingenieria En Telecomunicaciones
  - Ingenieria Civil
  - Seguridad De Redes De Informacion


In [6]:
# Generar todas las combinaciones de habilidades blandas (2^7 = 128 combinaciones)
def generar_combinaciones_habilidades_blandas(num_habilidades=7):
    """
    Genera todas las 2^n combinaciones posibles de habilidades blandas.
    
    Returns:
        list: Lista de tuplas, donde cada tupla es una combinación binaria de 7 elementos
              Ejemplo: (0,0,0,0,0,0,0), (0,0,0,0,0,0,1), ..., (1,1,1,1,1,1,1)
    """
    return list(product([0, 1], repeat=num_habilidades))

# Generar combinaciones
combinaciones = generar_combinaciones_habilidades_blandas(len(habilidades_blandas_config))

print(f"Total de combinaciones generadas: {len(combinaciones)}")
print(f"\nPrimeras 10 combinaciones:")
for i, comb in enumerate(combinaciones[:10]):
    print(f"  {i}: {comb}")
print("  ...")
print(f"\nÚltimas 10 combinaciones:")
for i, comb in enumerate(combinaciones[-10:], start=len(combinaciones)-10):
    print(f"  {i}: {comb}")

Total de combinaciones generadas: 128

Primeras 10 combinaciones:
  0: (0, 0, 0, 0, 0, 0, 0)
  1: (0, 0, 0, 0, 0, 0, 1)
  2: (0, 0, 0, 0, 0, 1, 0)
  3: (0, 0, 0, 0, 0, 1, 1)
  4: (0, 0, 0, 0, 1, 0, 0)
  5: (0, 0, 0, 0, 1, 0, 1)
  6: (0, 0, 0, 0, 1, 1, 0)
  7: (0, 0, 0, 0, 1, 1, 1)
  8: (0, 0, 0, 1, 0, 0, 0)
  9: (0, 0, 0, 1, 0, 0, 1)
  ...

Últimas 10 combinaciones:
  118: (1, 1, 1, 0, 1, 1, 0)
  119: (1, 1, 1, 0, 1, 1, 1)
  120: (1, 1, 1, 1, 0, 0, 0)
  121: (1, 1, 1, 1, 0, 0, 1)
  122: (1, 1, 1, 1, 0, 1, 0)
  123: (1, 1, 1, 1, 0, 1, 1)
  124: (1, 1, 1, 1, 1, 0, 0)
  125: (1, 1, 1, 1, 1, 0, 1)
  126: (1, 1, 1, 1, 1, 1, 0)
  127: (1, 1, 1, 1, 1, 1, 1)


In [8]:
# Crear DataFrames de usuarios sintéticos
def crear_usuarios_sinteticos(carreras, combinaciones_blandas):
    """
    Crea un DataFrame con todos los usuarios sintéticos.
    
    Args:
        carreras: lista de nombres de carreras
        combinaciones_blandas: lista de tuplas con combinaciones (0,1) de habilidades
    
    Returns:
        pd.DataFrame: DataFrame con columnas de usuario_id, carrera, y 7 habilidades blandas
    """
    usuarios_data = []
    usuario_id = 0
    
    for carrera in carreras:
        for combinacion in combinaciones_blandas:
            usuario_data = {
                'usuario_id': usuario_id,
                'carrera': carrera,
                'Gestión': combinacion[0],
                'Comunicación efectiva': combinacion[1],
                'Liderazgo': combinacion[2],
                'Trabajo en equipo': combinacion[3],
                'Ética profesional': combinacion[4],
                'Responsabilidad social': combinacion[5],
                'Aprendizaje autónomo': combinacion[6]
            }
            usuarios_data.append(usuario_data)
            usuario_id += 1
    
    return pd.DataFrame(usuarios_data)

# Crear el DataFrame de usuarios sintéticos
df_usuarios_sinteticos = crear_usuarios_sinteticos(carreras_disponibles, combinaciones)

print(f"✓ DataFrame de usuarios sintéticos creado")
print(f"  Dimensiones: {df_usuarios_sinteticos.shape}")
print(f"  {df_usuarios_sinteticos.shape[0]} usuarios × {df_usuarios_sinteticos.shape[1]} columnas")

✓ DataFrame de usuarios sintéticos creado
  Dimensiones: (3072, 9)
  3072 usuarios × 9 columnas


In [12]:
print(f"\nPrimeros 10 usuarios:")
df_usuarios_sinteticos.head(10)


Primeros 10 usuarios:


,usuario_id,carrera,Gestión,Comunicación efectiva,Liderazgo,Trabajo en equipo,Ética profesional,Responsabilidad social,Aprendizaje autónomo
0,0,Licenciatura Administracion De Empresas,0,0,0,0,0,0,0
1,1,Licenciatura Administracion De Empresas,0,0,0,0,0,0,1
2,2,Licenciatura Administracion De Empresas,0,0,0,0,0,1,0
3,3,Licenciatura Administracion De Empresas,0,0,0,0,0,1,1
4,4,Licenciatura Administracion De Empresas,0,0,0,0,1,0,0
5,5,Licenciatura Administracion De Empresas,0,0,0,0,1,0,1
6,6,Licenciatura Administracion De Empresas,0,0,0,0,1,1,0
7,7,Licenciatura Administracion De Empresas,0,0,0,0,1,1,1
8,8,Licenciatura Administracion De Empresas,0,0,0,1,0,0,0
9,9,Licenciatura Administracion De Empresas,0,0,0,1,0,0,1


In [14]:
print(f"\nÚltimos 10 usuarios:")
df_usuarios_sinteticos.tail(10)


Últimos 10 usuarios:


,usuario_id,carrera,Gestión,Comunicación efectiva,Liderazgo,Trabajo en equipo,Ética profesional,Responsabilidad social,Aprendizaje autónomo
3062,3062,Seguridad De Redes De Informacion,1,1,1,0,1,1,0
3063,3063,Seguridad De Redes De Informacion,1,1,1,0,1,1,1
3064,3064,Seguridad De Redes De Informacion,1,1,1,1,0,0,0
3065,3065,Seguridad De Redes De Informacion,1,1,1,1,0,0,1
3066,3066,Seguridad De Redes De Informacion,1,1,1,1,0,1,0
3067,3067,Seguridad De Redes De Informacion,1,1,1,1,0,1,1
3068,3068,Seguridad De Redes De Informacion,1,1,1,1,1,0,0
3069,3069,Seguridad De Redes De Informacion,1,1,1,1,1,0,1
3070,3070,Seguridad De Redes De Informacion,1,1,1,1,1,1,0
3071,3071,Seguridad De Redes De Informacion,1,1,1,1,1,1,1
